### 나이브 베이지안 분류기 (Naive Bayes)
Naive : 어리숙한

즉, 모든 사건을 독립적이라고 가정하는, 조건부 확률 기반 방법

(조건부 확률 : 어떤 사건 B가 일어났을 때 사건 A가 발생할 확률)

ex. 날씨에 따라 필드에 나갈 지 말지를 결정하려고 하는 문제

오늘 날씨 : 비가 오고, 온도가 적당하고, 습도가 높고, 바람이 분다

##### 분류 기준
> MAP(Maximum A Posteriori)

        사후 확률을 계산해 더 높은 확률을 가지는 것을 정답으로 분류
##### 종류
1. 가우시안 나이브 베이즈 (GaussianNB)
2. 다항분포 나이브 베이즈 (MultimonialNB())
   
    이산적인 값에 사용, 하나의 특성이 여러 종류로 나뉘는 경우

    ex. 스팸메일 - 각 단어가 스팸 메일에서 나타나는 확률을 계산하고 모든 단어의 확률을 더해서 하나의 메일이 스팸 확률을 계산하는 방법이 바로 다항분포 베이지안 분류 방법이다.

3. 베르누이 나이브 베이즈(이항분포) (BernoulliNB())
   
    이산적인 값에 사용, 모든 특성이 두 종류로만 나뉘는 경우

In [30]:
"""
이산적인 데이터 - MultinomiaNB
"""
import pandas as pd
from sklearn.naive_bayes import MultinomialNB

# 자료 읽기
data = pd.read_csv('./weather.csv', sep=',', engine='python')
print(data.head())

# Naive Bayes 모듈 중 다항분포 나이브 베이즈 가져와 인스턴스화 하기 
multinomial_model = MultinomialNB()

# Category 문자 데이터를 숫자 category로 변환 (Map 활용)
# mapping dictionary 만들기
outlook_dic = {"overcast":0, "rainy":1, "sunny":2}
temperature_dic = {"cool":0, "hot":1, "mild":2}
humidity_dic = {"high":0, "normal":1}
windy_dic = {False:0, True:1}

# dictionary 활용해 data mapping
data["outlook"] = data["outlook"].map(outlook_dic)
data["temperature"] = data["temperature"].map(temperature_dic)
data["humidity"] = data["humidity"].map(humidity_dic)
data["windy"] = data["windy"].map(windy_dic)

multinomial_model.fit(data.iloc[:, :4], data["play"])

multinomial_model.predict([[0,1,0,0]])

    outlook temperature humidity  windy play
0  overcast         hot     high  False  yes
1  overcast        cool   normal   True  yes
2  overcast        mild     high   True  yes
3  overcast         hot   normal  False  yes
4     rainy        mild     high  False  yes


/Users/emilynan/Desktop/Daeyeon/ICT_AI_Class/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


array(['yes'], dtype='<U3')

In [27]:
"""
연속적인 데이터 - GaussianNB
"""
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

iris = load_iris()

# 자료 준비
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df["species"] = iris.target

# 가우시안 모델 인스턴스화
gaussian_model = GaussianNB()

# 데이터를 학습데이터와 테스트 데이터로 분할 후 학습 데이터로 학습
X_train, X_test, y_train, y_test = train_test_split(iris_df.iloc[:,:4], iris_df["species"], test_size = 0.33)
gaussian_model.fit(X_train, y_train)

# 테스트 데이터로 성능 평가
print(classification_report(y_test, gaussian_model.predict(X_test)))
print(confusion_matrix(y_test, gaussian_model.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       0.93      1.00      0.97        14
           2       1.00      0.94      0.97        18

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50

[[18  0  0]
 [ 0 14  0]
 [ 0  1 17]]


### 결정 트리
아직 일어나지 않은 일에 대하여 미리 판단해야 하는 종류의 문제에 적용

데이터들의 분포를 분류하여 결과를 예측하는 알고리즘

(어떤 모양(선형, 비선형,,)의 데이터가 들어와도 풀 수 있음)

Classification과 regression을 수행하는 분류 알고리즘

##### 장점
White-box 모델 (내부가 ㅌ투명해 결과가 나오는 이유를 설명할 수 있다.)

시각화될 수 있어 이해가 쉽고, 해석이 간단하며, 전처리 절차가 간단하다.

##### 단점
불안정 (데이터가 조금만 변해도 아예 다른 decision tree가 생성될 수 있다)

XOR 표현 못함

##### 영역 분할
정보획득이 많은 방향

영역의 순도(homogeneity)는 증가, 불순도(Impurity)는 감소하는 방향으로 분할

##### 순도를 계산하는 방법
1. Entropy에 의한 방법
    엔트로피 = 무질서도 = 정보력이 낮음 (엔트로피 값이 높다 == 무질서 하다 == 애매하다 == 정보력이 낮다)

    즉, 엔트로피를 최소화시키는 방향으로 사용
2. Gini index(불순도)에 의한 방법
3. MSE(Mean square error)에 의한 방법

In [31]:
import pandas as pd
from sklearn import tree

# 데이터 준비
data = pd.read_csv("./weather.csv", sep=",", engine="python")

# mapping dictionary 만들기
outlook_dic = {"overcast":0, "rainy":1, "sunny":2}
temperature_dic = {"cool":0, "hot":1, "mild":2}
humidity_dic = {"high":0, "normal":1}
windy_dic = {False:0, True:1}

# dictionary 활용해 data mapping
data["outlook"] = data["outlook"].map(outlook_dic)
data["temperature"] = data["temperature"].map(temperature_dic)
data["humidity"] = data["humidity"].map(humidity_dic)
data["windy"] = data["windy"].map(windy_dic)

# 결정 트리 분류기 인스턴스 생성
clf = tree.DecisionTreeClassifier()

# 결정 트리 분류기 학습
clf.fit(data.iloc[:, :4], data["play"])

# 예측 (오늘 같은 날씨에는 필드에 나갈까?)
# 맑고, 온도가 적당하고, 습도가 높고, 바람이 분다
clf.predict([[2,2,0,1]])

/Users/emilynan/Desktop/Daeyeon/ICT_AI_Class/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array(['no'], dtype=object)

In [6]:
import pandas as pd
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split


iris = load_iris()

iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df["species"] = iris.target
# print(iris_df)

# 결정 트리 분류기 인스턴스 생성
clf = tree.DecisionTreeClassifier()

# 데이터를 학습데이터와 테스트 데이터로 분할 후 학습 데이터로 학습
X_train, X_test, y_train, y_test = train_test_split(iris_df.iloc[:,:4], iris_df["species"], test_size = 0.33)
clf.fit(X_train, y_train)

# 테스트 데이터로 성능 평가
print(classification_report(y_test, clf.predict(X_test)))
print(confusion_matrix(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.94      0.97        18
           2       0.93      1.00      0.96        13

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50

[[19  0  0]
 [ 0 17  1]
 [ 0  0 13]]
